In [1]:
import sys; sys.path

['e:\\research\\Prof.Chen\\20200110\\data\\JD data',
 'c:\\Users\\zexin\\.vscode\\extensions\\ms-python.python-2020.9.114305\\pythonFiles',
 'c:\\Users\\zexin\\.vscode\\extensions\\ms-python.python-2020.9.114305\\pythonFiles\\lib\\python',
 'C:\\Users\\zexin\\Anaconda3\\envs\\research1.0\\python37.zip',
 'C:\\Users\\zexin\\Anaconda3\\envs\\research1.0\\DLLs',
 'C:\\Users\\zexin\\Anaconda3\\envs\\research1.0\\lib',
 'C:\\Users\\zexin\\Anaconda3\\envs\\research1.0',
 '',
 'C:\\Users\\zexin\\AppData\\Roaming\\Python\\Python37\\site-packages',
 'C:\\Users\\zexin\\Anaconda3\\envs\\research1.0\\lib\\site-packages',
 'C:\\Users\\zexin\\Anaconda3\\envs\\research1.0\\lib\\site-packages\\win32',
 'C:\\Users\\zexin\\Anaconda3\\envs\\research1.0\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\zexin\\Anaconda3\\envs\\research1.0\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\zexin\\Anaconda3\\envs\\research1.0\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\zexin\\.ipython']

In [2]:
# Set working directory
import os

path = os.getcwd()

print(path)
os.chdir('E:/research/Prof.Chen/20200110/data/JD data')

print(os.getcwd())

e:\research\Prof.Chen\20200110\data\JD data
E:\research\Prof.Chen\20200110\data\JD data


In [3]:
import numpy as np
import pandas as pd
import datetime as dt
from numpy import array
from numpy import argmax
import csv
# Modeling
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import scipy 
import scipy.stats as ss

# Optimization Solver
import cvxpy as cp


In [4]:
# 'skus' table
skus = pd.read_csv('JD_sku_data.csv')
# 'users' table
users = pd.read_csv('JD_user_data.csv')
# 'orders' table
orders = pd.read_csv('JD_order_data.csv')

In [5]:
# Basic information of skus dataset
print(skus.shape, skus.head())

(31868, 7)        sku_ID  type    brand_ID attribute1 attribute2 activate_date  \
0  a234e08c57     1  c3ab4bf4d9        3.0       60.0           NaN   
1  6449e1fd87     1  1d8b4b4c63        2.0       50.0           NaN   
2  09b70fcd83     2  eb7d2a675a        3.0       70.0           NaN   
3  acad9fed04     2  9b0d3a5fc6        3.0       70.0           NaN   
4  2fa77e3b4d     2  b681299668          -          -           NaN   

  deactivate_date  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN  


In [6]:
# Basic information of users dataset
print(users.shape, users.head())
users.head()

(457298, 10)       user_ID  user_level first_order_month  plus gender    age  \
0  000089d6a6           1           2017-08     0      F  26-35   
1  0000babd1f           1           2018-03     0      U      U   
2  0000bc018b           3           2016-06     0      F   >=56   
3  0000d0e5ab           3           2014-06     0      M  26-35   
4  0000dce472           3           2012-08     1      U      U   

  marital_status  education  city_level  purchase_power  
0              S          3           4               3  
1              U         -1          -1              -1  
2              M          3           2               3  
3              M          3           2               2  
4              U         -1          -1              -1  


,user_ID,user_level,first_order_month,plus,gender,age,marital_status,education,city_level,purchase_power
0,000089d6a6,1,2017-08,0,F,26-35,S,3,4,3
1,0000babd1f,1,2018-03,0,U,U,U,-1,-1,-1
2,0000bc018b,3,2016-06,0,F,>=56,M,3,2,3
3,0000d0e5ab,3,2014-06,0,M,26-35,M,3,2,2
4,0000dce472,3,2012-08,1,U,U,U,-1,-1,-1


In [7]:
# Basic information of orders dataset
print(orders.shape, orders.head()) 
print('The number of unique product purchased:',len(orders['sku_ID'].unique()))



(549989, 17)      order_ID     user_ID      sku_ID  order_date             order_time  \
0  d0cf5cc6db  0abe9ef2ce  581d5b54c1  2018-03-01  2018-03-01 17:14:25.0   
1  7444318d01  33a9e56257  067b673f2b  2018-03-01  2018-03-01 11:10:40.0   
2  f973b01694  4ea3cf408f  623d0a582a  2018-03-01  2018-03-01 09:13:26.0   
3  8c1cec8d4b  b87cb736cb  fc5289b139  2018-03-01  2018-03-01 21:29:50.0   
4  d43a33c38a  4829223b6f  623d0a582a  2018-03-01  2018-03-01 19:13:37.0   

   quantity  type promise  original_unit_price  final_unit_price  \
0         1     2       -                 89.0              79.0   
1         1     1       2                 99.9              53.9   
2         1     1       2                 78.0              58.5   
3         1     1       2                 61.0              35.0   
4         1     1       1                 78.0              53.0   

   direct_discount_per_unit  quantity_discount_per_unit  \
0                       0.0                        10.0   
1  

In [8]:
## groupby by sku_ID
## https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html
orders.groupby(['sku_ID']).size().nlargest(5).reset_index(name='top5')

,sku_ID,top5
0,068f4481b3,23655
1,3c79df1d80,14463
2,a9109972d1,12176
3,623d0a582a,10175
4,8dc4a01dec,9891


In [9]:
## Only select the product with the most samples
orders_single = orders.loc[orders['sku_ID'] == '068f4481b3']

print('The sample size is:',orders_single.shape, orders_single.describe())
print('The number of unique customers purchase this product:',len(orders_single['user_ID'].unique()), ', which is', len(orders_single['user_ID'].unique()) == orders_single.shape[0], 'equal to the number of samples.')

The sample size is: (23655, 17)            quantity     type  original_unit_price  final_unit_price  \
count  23655.000000  23655.0         23655.000000      23655.000000   
mean       1.089368      1.0           297.155347        218.220955   
std        1.445569      0.0             9.814908         15.174430   
min        1.000000      1.0           119.200000          2.000000   
25%        1.000000      1.0           298.000000        208.000000   
50%        1.000000      1.0           298.000000        216.000000   
75%        1.000000      1.0           298.000000        228.000000   
max      139.000000      1.0           745.000000        745.000000   

       direct_discount_per_unit  quantity_discount_per_unit  \
count              23655.000000                     23655.0   
mean                  68.899472                         0.0   
std                   25.120224                         0.0   
min                    0.000000                         0.0   
25%          

In [10]:
## Only select columns: order_ID, user_ID(customer feature ID, key), quantity(demand), final_unit_price(price), final_total_price(revenue)
orders_single_clean = orders_single[['user_ID', 'quantity', 'final_unit_price']]
print('The unmber of unique prices is', len(orders_single_clean['final_unit_price'].unique()), '. The prices distribute as',  orders_single_clean['final_unit_price'].describe())
print(orders_single_clean.head())
## aggregate quantity and final_total_price across the unique customer
orders_single_clean_agg = orders_single_clean
orders_single_clean_agg['final_total_price'] = orders_single_clean['quantity'] * orders_single_clean['final_unit_price']
orders_single_clean_agg = orders_single_clean_agg.groupby(by=["user_ID"], dropna=True).sum()
print('The sample size after aggregation is',orders_single_clean_agg.shape)
orders_single_clean_agg['final_unit_price'] = orders_single_clean_agg['final_total_price'] / orders_single_clean_agg['quantity']
print(orders_single_clean_agg.head(3))
print('The unmber of unique prices after aggregation is', len(orders_single_clean_agg['final_unit_price'].unique()), '. The prices distribute as',  orders_single_clean_agg['final_unit_price'].describe())

The unmber of unique prices is 188 . The prices distribute as count    23655.000000
mean       218.220955
std         15.174430
min          2.000000
25%        208.000000
50%        216.000000
75%        228.000000
max        745.000000
Name: final_unit_price, dtype: float64
       user_ID  quantity  final_unit_price
9   2021a86702         1             208.0
34  3f399e9323         1             208.0
78  f83141d477         1             198.0
83  28ac8c9203         1             208.0
98  12f610851e         1             198.0
The sample size after aggregation is (22996, 3)
            quantity  final_unit_price  final_total_price
user_ID                                                  
0009796280         1             245.0              245.0
001282c65e         1             208.0              208.0
0012f95e63         1             208.0              208.0
The unmber of unique prices after aggregation is 203 . The prices distribute as count    22996.000000
mean       218.238303
std

In [11]:
# Data Cleansing
# 

In [12]:
# Feature Engineering
# The selected user features
# normal(i.e.marital_status	education	city_level	purchase_power) + sensitive(i.e. gender)
users_trim = users[['user_ID', 'user_level','age',	'marital_status', 'education',	'city_level', 'purchase_power', 'gender']]


In [13]:
#### Remove all uncatorized rows
#### Gender : select F or M and remove U 

users_clean = users_trim.loc[(users_trim['gender'] == 'F') | (users_trim['gender'] == 'M')] 
print('The user sample size for female and male only is', users_clean.shape)
print(users_clean.head(3))


The user sample size for female and male only is (399981, 8)
      user_ID  user_level    age marital_status  education  city_level  \
0  000089d6a6           1  26-35              S          3           4   
2  0000bc018b           3   >=56              M          3           2   
3  0000d0e5ab           3  26-35              M          3           2   

   purchase_power gender  
0               3      F  
2               3      F  
3               2      M  


In [14]:
#### age (<=15: less than or equal to 15 years old, 16-25: 16 to 25 years old, 26-35: 26 to 35 years old, 36-45: 36 to 45 years old, 46-55: 46 to 55 years old, >=56: greater than or equal to 56 years old, U: unknown); 
#### marriage – user’s marital status (M: Married, S: Single, U: Unknown)
#### education – user’s education level (1: less than high school, 2: high school diploma or equivalent, 3: Bachelor’s degree, 4: post-graduate degree, −1: unknown); 
#### purchase_power – user’s estimated purchase power (ranging from 1 to 5 with 1 being the highest purchase power; −1 if there is no estimation) 
#### city_level, which takes on values ranging between 1 and 5: level 1 corresponds to highly industrialized cities such as Beijing and Shanghai; level 2 cities correspond to provincial capitals; level 3 to 5 cities are smaller cities; if there are no data then the value is −1. 
#### user_level takes on a value of 0, 1, 2, 3, or 4, where a higher user_level is associated with a higher total purchase value in the past. For users who are enterprise users (e.g., small shops in rural areas or small businesses), the corresponding user_level takes on a value of 10. However, for first-time purchasers, their user_level takes on the value −1 

# users_clean = users_clean.drop(users_trim[users_trim['age'] == 'U'].index, inplace = False) 
users_clean = users_clean.loc[(users_clean['age'] != 'U') & (users_clean['marital_status'] != 'U') & (users_clean['education'] != -1) & (users_clean['purchase_power'] != -1) & (users_clean['city_level'] != -1) & (users_clean['user_level'] != -1)]
print('The user sample size for all identified groups is', users_clean.shape)
print(users_clean.head(3))

The user sample size for all identified groups is (316230, 8)
      user_ID  user_level    age marital_status  education  city_level  \
0  000089d6a6           1  26-35              S          3           4   
2  0000bc018b           3   >=56              M          3           2   
3  0000d0e5ab           3  26-35              M          3           2   

   purchase_power gender  
0               3      F  
2               3      F  
3               2      M  


In [15]:
## Feature transformation and encoding
label_encoder = LabelEncoder()
## Gender: Female = 1, Male = 2
users_clean['gender'] = label_encoder.fit_transform(users_clean['gender']) + 1 
print(users_clean['gender'])

0         1
2         1
3         2
5         1
6         1
         ..
457286    1
457291    1
457292    1
457294    2
457295    2
Name: gender, Length: 316230, dtype: int32


In [16]:
## age
users_clean['age'] = label_encoder.fit_transform(users_clean['age']) + 1
print(users_clean['age']) 

0         2
2         6
3         2
5         2
6         2
         ..
457286    4
457291    2
457292    2
457294    2
457295    3
Name: age, Length: 316230, dtype: int32


In [17]:
## marital_status
users_clean['marital_status'] = label_encoder.fit_transform(users_clean['marital_status']) + 1
print(users_clean['marital_status'])

0         2
2         1
3         1
5         1
6         1
         ..
457286    1
457291    1
457292    2
457294    2
457295    1
Name: marital_status, Length: 316230, dtype: int32


In [18]:
# The product we select
# The columns selected in the order dataset 

In [19]:
# Combine 2 dfs
orders_single_clean = orders_single[['user_ID', 'quantity', 'final_unit_price']]
print('The size of the cleaned data from a single product is', orders_single_clean.shape)
print(orders_single_clean.head())
orders_users = pd.merge(orders_single_clean, users_clean, on='user_ID', how='left')
print('The size of the merged data from a single product and corresponding unique users is', orders_users.shape)
print(orders_users.head(3), orders_users.columns.values)
print(orders_users.describe())
print

The size of the cleaned data from a single product is (23655, 3)
       user_ID  quantity  final_unit_price
9   2021a86702         1             208.0
34  3f399e9323         1             208.0
78  f83141d477         1             198.0
83  28ac8c9203         1             208.0
98  12f610851e         1             198.0
The size of the merged data from a single product and corresponding unique users is (23655, 10)
      user_ID  quantity  final_unit_price  user_level  age  marital_status  \
0  2021a86702         1             208.0         NaN  NaN             NaN   
1  3f399e9323         1             208.0         2.0  1.0             2.0   
2  f83141d477         1             198.0         2.0  1.0             1.0   

   education  city_level  purchase_power  gender  
0        NaN         NaN             NaN     NaN  
1        3.0         2.0             2.0     1.0  
2        2.0         3.0             4.0     1.0   ['user_ID' 'quantity' 'final_unit_price' 'user_level' 'age'
 'ma

<function print>

In [20]:
# Fit linear demand function
## data preprocessing
# drop rows with missing values
orders_users.dropna(inplace=True)
# summarize the shape of the data with missing rows removed
print(orders_users.shape)
### create training and testing datasets
from sklearn.model_selection import train_test_split

column_values = ['final_unit_price', 'user_level', 'age', 'marital_status', 'education', 'city_level', 'purchase_power', 'gender']
X_train, X_test, y_train, y_test = train_test_split(orders_users[column_values], orders_users['quantity'], test_size=0.88)
# train_index_values = X_train['user_ID']
# print(len(train_index_values))
X_train.columns = column_values
print(X_train.shape, X_test.shape)
print(X_train.head(3))
### standardize the training data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
X_std_train = MinMaxScaler(feature_range = (1,5), copy = True).fit_transform(X_train)
X_std_test = MinMaxScaler(feature_range = (1,5), copy = True).fit_transform(X_test)
X_std_train_df = pd.DataFrame(data = X_std_train,  
                  # index = train_index_values,  
                  columns = column_values) 
X_std_test_df = pd.DataFrame(data = X_std_test,  
                  # index = test_index_values,  
                  columns = column_values) 
print(X_std_train_df.head(3))
print(X_std_test_df.head(3))

(19031, 10)
(3044, 8) (15987, 8)
       final_unit_price  user_level  age  marital_status  education  \
18396             225.0         4.0  2.0             1.0        4.0   
4072              208.0         4.0  2.0             1.0        4.0   
23380             216.0         4.0  3.0             1.0        4.0   

       city_level  purchase_power  gender  
18396         3.0             2.0     1.0  
4072          2.0             2.0     1.0  
23380         2.0             3.0     1.0  
   final_unit_price  user_level  age  marital_status  education  city_level  \
0          3.918519         5.0  1.8             1.0        5.0         3.0   
1          3.666667         5.0  1.8             1.0        5.0         2.0   
2          3.785185         5.0  2.6             1.0        5.0         2.0   

   purchase_power  gender  
0             2.0     1.0  
1             2.0     1.0  
2             3.0     1.0  
   final_unit_price  user_level  age  marital_status  education  city_level  

In [21]:
## Colinerality Detection
eigenvalues, eigenvectors = np.linalg.eig(pd.DataFrame(X_std_train).corr())
pd.Series(eigenvalues).sort_values()
#### in this DATASET, the eigenvalueS ARE NOT TOO SMALL small which does not show evidence of collinearity. 

1    0.448662
2    0.616915
6    0.821875
7    0.862657
5    0.995529
4    1.015482
3    1.192836
0    2.046045
dtype: float64

In [28]:
## Data
## normalize
column_values = column_values[1:]
orders_users_X = orders_users[column_values]
orders_users_X_std = minmax_scale(orders_users_X, feature_range=(1,5), axis=0, copy=True)
orders_users_X_std_df = pd.DataFrame( orders_users_X_std,  
                  columns = column_values) 